### Imports des modules :

In [19]:
import pandas as pd
import xarray as xr
import numpy as np
from datetime import datetime

### Lecture du fichier :

In [20]:
df = pd.read_excel("../data/bresil.xlsx", keep_default_na=False)

# on récupère les données
jour = df["jour"].tolist()
heure = df["heure"].tolist()
# 
mission = df["mission"].tolist()
num_station = df["num_station"].tolist()
lat = df["lat"].tolist()
lon = df["lon"].tolist()
profondeur = df["profondeur"].tolist()
chla = df["chla"].tolist()
mes = df["mes"].tolist()
doc = df["doc"].tolist()
poc = df["poc"].tolist()

# On compte le nombre de lignes du fichier XLSX
row = len(df) # 3334

### Traitement des données :

In [21]:
# On récupère seulement la date
jour = [str(jour[i])[:10] for i in range(len(jour))]
heure_str = [str(h) for h in heure]
# si il y a pas d'hreures, on met 00:00:00
for i in range(len(heure_str)):
    if heure_str[i] == "":
        heure_str[i] = "00:00:00"
    else:
        heure_str[i] = heure_str[i]

# On convertit la date avec jour et heure_str en ce modèle : 1995-10-11T15:25:00Z
liste_combinee = []
for i in range(len(jour)):
    date_str = jour[i] + "T" + heure_str[i] + "Z"
    date = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    liste_combinee.append(date)
    
# Pour chla, mes, doc, poc on les definit comme float et si il y a pas de valeur, on met NaN
for i in range(len(chla)):
    if chla[i] == "":
        chla[i] = np.nan
    else:
        chla[i] = float(chla[i])

for i in range(len(mes)):
    if mes[i] == "":
        mes[i] = np.nan
    else:
        mes[i] = float(mes[i])

for i in range(len(doc)):
    if doc[i] == "":
        doc[i] = np.nan
    else:
        doc[i] = float(doc[i])

for i in range(len(poc)):
    if poc[i] == "":
        poc[i] = np.nan
    else:
        poc[i] = float(poc[i])

### Remplissage du Dataset :

In [22]:
# Les variables de la dataset


# Les attributs de la dataset
attrs = {
    'title': 'Données de la campagne Bresil',
    'institution': "Laboratoire d'Océanologie et de Géosciences",
    'Conventions': 'CF-1.6',
    'source': 'CNRS',
}
    

ds = xr.Dataset(
    {
        "mission": ("row", mission),
        "num_station": ("row", num_station),
        
        "lat": ("row", lat, 
                    {'units': 'degrees_north',
                     'long_name': 'Latitude',
                     'standard_name': 'latitude',
                     'axis': 'Y'}),
        
        "lon": ("row", lon,
                    {'units': 'degrees_east',
                     'long_name': 'Longitude',
                     'standard_name': 'longitude',
                     'axis': 'X'}),
        
        "time" : ("row", liste_combinee, 
                    {'long_name': 'Start Time',
                     'standard_name': 'time',
                     'axis': 'T'}),
        
        "profondeur": ("row", profondeur,
                        {'units': 'm',
                         'long_name': 'Profondeur'}),
        
        "chla": ("row", chla,
                    {'units': 'µg.L-1',
                     'long_name': 'Chlorophylle a'}),
        
        "mes": ("row", mes,
                    {'units': 'µg.L-1',
                     'long_name': 'Matière en suspension'}),
        
        "doc": ("row", doc,
                    {'units': 'µmol.L-1',
                     'long_name': 'Dissolved organic carbon'}),
        
        "poc": ("row", poc,
                    {'units': 'µmol.L-1',
                     'long_name': 'Particulate organic carbon'}),
},
    attrs=attrs,   
)

In [23]:
print(ds)

<xarray.Dataset>
Dimensions:      (row: 3334)
Dimensions without coordinates: row
Data variables:
    mission      (row) <U24 'Nomad_amt1' 'Nomad_amt1' ... 'SEPETIBA-ILHAGRANDE'
    num_station  (row) <U28 'S95' 'S96' 'S97' ... 'S-IL32' 'S-IL33' 'S-IL34'
    lat          (row) float64 -11.47 -15.58 -19.64 ... -23.14 -23.14 -23.07
    lon          (row) float64 -31.55 -33.45 -35.36 ... -44.63 -44.57 -44.43
    time         (row) datetime64[ns] 1995-10-11T15:25:00 ... 2022-09-15T11:5...
    profondeur   (row) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    chla         (row) float64 -999.0 -999.0 -999.0 ... 0.6852 1.294 0.8704
    mes          (row) float64 nan nan nan nan nan ... 4.413 10.05 11.43 3.207
    doc          (row) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    poc          (row) float64 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    title:        Données de la campagne Bresil
    institution:  Laboratoire d'Océanologie et de Géoscien

### Création du fichier netCDF :

In [24]:
ds.to_netcdf("../bresil.nc") #Supprimer le fichier avant de le refaire